#  Spatial Filtering

# Question 1

In [4]:
import cv2
import matplotlib
%matplotlib
import matplotlib.pyplot as plt
import numpy as np
import math

Using matplotlib backend: Qt5Agg


In [1]:
def convolution(img,kernel=0,kernel_type='none'):
    output = np.zeros_like(img)           
    # Add zero padding to the input image
    kernel_x = kernel.shape[0]
    kernel_y = kernel.shape[1]
    image_padded = np.zeros((img.shape[0] + (kernel_x-1) , img.shape[1] + kernel_y-1))   
    image_padded[1:-1, 1:-1] = img
    #print(image_padded)
    if kernel_type == 'median':
        for x in range(kernel_x//2,img.shape[0]+kernel_x//2):     
            for y in range(kernel_y//2,img.shape[1]+kernel_y//2):
               # print(x-kernel_x//2,y-kernel_y//2)
                # element wise multiplication of the image pixels and kernel
                array = image_padded[x-kernel_x//2:x+kernel_x//2+1,y-kernel_y//2:y+kernel_y//2+1]
                output[x-kernel_x//2,y-kernel_y//2] = np.median(array)
    else:        
        for x in range(kernel_x//2,img.shape[0]+kernel_x//2):     
            for y in range(kernel_y//2,img.shape[1]+kernel_y//2):
               # print(x-kernel_x//2,y-kernel_y//2)
                # element wise multiplication of the image pixels and kernel
                output[x-kernel_x//2,y-kernel_y//2]=int(((kernel*image_padded[x-kernel_x//2:x+kernel_x//2+1,y-kernel_y//2:y+kernel_y//2+1]).sum()))        
    return output

In [18]:
def imnoise(img,noise_type,amt=0):
    if(noise_type == 'gaussian'):
        row,col = img.shape
        mean = 5
        var = 15
        sigma = var**0.5
        gauss_noise = np.random.normal(mean,sigma,(row,col))
        gauss_noise = gauss_noise.reshape(row,col)
        noisy_img = img + gauss_noise
        return noisy_img
    elif noise_type == "s&p":
        row,col= img.shape
        s_vs_p = 0.5
        amount = amt
        out = np.copy(img)
      # Salt mode
        num_salt = np.ceil((amount/2) * img.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))
                  for i in img.shape]
        out[coords] = 255

      # Pepper mode
        num_pepper = np.ceil((amount/2)* img.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in img.shape]
        out[coords] = 0
        return out
    else:
        return

In [19]:
img = cv2.imread("/media/shubham/New Volume/DAIICT/2nd sem/DIP/assignment4/lena.jpg",0)
kernel = np.array([[0,-1,0],[-1,5,-1],[0,-1,0]])
noise_remove_kernel = np.array([[1,1,1],[1,1,1],[1,1,1]])*(1/9) #mean filter
given_kernel = np.reshape(kernel,(3,3))
kernel = np.zeros(given_kernel.shape)

#flipping kernel
for i in range(0,kernel.shape[0]):
    for j in raange(0,kernel.shape[1]):
        kernel[i,j] = given_kernel[-i+(given_kernel.shape[0]-1),-j+(given_kernel.shape[1]-1)]
# image after adding noise
density = [0.05,0.1,0.2 ]
for dens in density:
    noisy_img  = imnoise(img,'s&p',dens)
    output_noisy = convolution(noisy_img,kernel,'median')
    #noisy image
    plt.figure()
    plt.subplot(121)
    plt.imshow(noisy_img)
    plt.gray()
    plt.show()

    #output of noisy image convolution
    plt.subplot(122)
    plt.imshow(output_noisy)
    plt.gray()
    plt.show()

# Question 2

In [154]:
img = cv2.imread("/media/shubham/New Volume/DAIICT/2nd sem/DIP/assignment4/lena.jpg",0)
img1 = np.copy(img)
plt.figure()
plt.gray()
plt.imshow(img1)
noise_remove_kernel = np.array([[1,1,1],[1,1,1],[1,1,1]])*(1/9) #mean filter
#noise_remove_kernel = np.array([[-1,2,1],[0,1,2],[-1,1,0]])
rows,cols = img.shape
r,c = noise_remove_kernel.shape

#shifting low frequencies to middle 
for i in range(0,rows):
     for j in range(0,cols):
        img1[i,j] = img1[i,j]*((-1)**(i+j))
# -------------------------------------------------------------------------------------        
#padding of image and kernel

img_padded = np.zeros((rows+r-1,cols+c-1))
#print("a_tempo shape ",a_temp.shape)
kernel_padded = np.zeros_like(img_padded)
print(img_padded.shape,kernel_padded.shape)

img_padded[1:-1,1:-1] = img1
m,n = kernel_padded.shape
m = m//2
n = n//2
print(m,n)
#kernel_padded[m-1:m-1+r,n-1:n-1+c]= noise_remove_kernel
kernel_padded[0:r,0:c] = noise_remove_kernel
for i in range(0,kernel_padded.shape[0]):
     for j in range(0,kernel_padded.shape[1]):
        kernel_padded[i,j] = kernel_padded[i,j]*((-1)**(i+j))
#kernel_padded[0:r,0:c] = noise_remove_kernel

#--------------------------------------------------------------------------



# 2D FFT of  image
img_padded = np.fft.fft(img_padded,axis=1)
img_padded = np.fft.fft(img_padded,axis=0)

# 2D fft of kernel

kernel_padded= np.fft.fft(kernel_padded,axis=1)
kernel_padded = np.fft.fft(kernel_padded,axis=0)

plt.figure()
plt.gray()
plt.subplot(121)
plt.imshow(img_padded.real)
plt.subplot(122)
plt.imshow(kernel_padded.real)
output = img_padded*kernel_padded

# ifft

output = np.fft.ifft(output,axis=1)
output = np.fft.ifft(output,axis=0)
for i in range(0,output.shape[0]):
     for j in range(0,output.shape[1]):
        output[i,j] = output[i,j]*((-1)**(i+j))
plt.figure()
plt.imshow(output.real)
plt.gray()
plt.show() 


(202, 202) (202, 202)
101 101


# Question 3

In [65]:
def cross_correlation(img,kernel):
    output = np.zeros_like(img)    
    output_img = np.copy(img)
    # Add zero padding to the input image
    kernel_x = kernel.shape[0]
    kernel_y = kernel.shape[1]
    image_padded = np.zeros((img.shape[0] + (kernel_x-1) , img.shape[1] + kernel_y-1))   
    print(image_padded.shape)
    image_padded[kernel_x//2:img.shape[0]+kernel_x//2, kernel_y//2:img.shape[1]+kernel_y//2] = img
    #print(image_padded)
    count = 0
    cp = 0
    for x in range(kernel_x//2,img.shape[0]):     
        #print(count)
        for y in range(kernel_y//2,img.shape[1]):
            temp_img = image_padded[x-kernel_x//2:x+kernel_x//2+1,y-kernel_y//2:y+kernel_y//2]
                
            img_mean= np.mean(temp_img)
            #print(temp_img)
           # print(img_mean)
            temp_img = temp_img - img_mean
            kernel_mean = np.mean(kernel)
            temp_kernel = kernel - kernel_mean
            temp_ans_num = (temp_kernel * temp_img ).sum()
            if(temp_ans_num == 'NaN'):
                continue
            #print("AAAAAAAAAAAAAAAAAAAAAAAa ",temp_ans_num)
            temp_ans_deno = (((temp_img**2).sum()) * ((temp_kernel**2).sum()))**0.5
            if(temp_ans_deno == 'NaN'):
                continue
            #print("temp_ans_deno ",temp_ans_deno)
            #print(temp_ans_num / temp_ans_deno)
            if(temp_ans_deno != 0):
                if(temp_ans_num / temp_ans_deno == 1.0):
                    output_img[x-kernel_x//2-kernel_x//2:x+kernel_x//2+1,y-kernel_y//2-kernel_y//2:y+kernel_y//2] = 0
                output[x-kernel_x//2,y-kernel_y//2] = temp_ans_num / temp_ans_deno
        count+=1
    
    return output_img

In [66]:
img1 = cv2.imread("/media/shubham/New Volume/DAIICT/2nd sem/DIP/assignment4/lab4text.png",0)
img2 = cv2.imread("/media/shubham/New Volume/DAIICT/2nd sem/DIP/assignment4/and.png",0)
print(img1.shape)
print(img2.shape)
c_c_mat = cross_correlation(img1,img2)
plt.figure()
plt.imshow(c_c_mat)
plt.gray()
plt.show()
print(c_c_mat)


(297, 892)
(25, 42)
(321, 933)
[[255 255 255 ..., 255 255 255]
 [255 255 255 ..., 255 255 255]
 [255 255 255 ..., 255 255 255]
 ..., 
 [255 255 255 ..., 255 255 255]
 [255 255 255 ..., 255 255 255]
 [255 255 255 ..., 255 255 255]]


In [29]:
a = np.array([[1,2,3,4,5],[4,5,6,9,2],[6,7,1,2,3],[4,4,4,5,6],[2,2,6,7,8]])
b= np.array([[1,2,3],[4,5,6],[6,7,8]])
print(a)
print(b)
ans = cross_correlation(a,b)
print(ans)
a_temp = np.zeros((a.shape[0]+b.shape[0]+1,a.shape[1]+b.shape[1]+1))
#print("a_tempo shape ",a_temp.shape)
p,q = b.shape
c = np.zeros_like(a_temp)
a_temp[p-1:-p+1,q-1:-q+1] = a
print(a_temp)
m,n = c.shape
m = m//2
n = n//2
#print(m,n)
c[m-1:-m+1,n-1:-n+1]= b
print(c)
print(a_temp.shape,c.shape)

[[1 2 3 4 5]
 [4 5 6 9 2]
 [6 7 1 2 3]
 [4 4 4 5 6]
 [2 2 6 7 8]]
[[1 2 3]
 [4 5 6]
 [6 7 8]]
(7, 7)
0
1
2
3
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 1 0]
 [0 0 0 0 0]]
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  2.  3.  4.  5.  0.  0.]
 [ 0.  0.  4.  5.  6.  9.  2.  0.  0.]
 [ 0.  0.  6.  7.  1.  2.  3.  0.  0.]
 [ 0.  0.  4.  4.  4.  5.  6.  0.  0.]
 [ 0.  0.  2.  2.  6.  7.  8.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]]
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  2.  3.  0.  0.  0.]
 [ 0.  0.  0.  4.  5.  6.  0.  0.  0.]
 [ 0.  0.  0.  6.  7.  8.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]]
(9, 9) (9, 9)
